# 50.040 Natural Language Processing (Fall 2024) Mini Project (40 Points)

**DUE DATE: 25 October 2024**

This homework will be graded by Chen Huang


# Personal Information (Fill before you start)

**STUDENT ID:** 1006184

**Name:** Atul Parida

**Students with whom you have discussed (if any):**

**Please also rename the final submitted pdf as ``miniproject_[NAME]_[STUDENTID].pdf``**

**-1 points if info not filled or file name not adjusted before submission, -40 points if you copy other's answer. We encourage discussion, but please do not copy without thinking.**

## [!] Please read this if your computer does not have GPUs.
### Free GPU Resources
We suggest that you run neural language models on machines with GPU(s). Google provides the free online platform [Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb), a research tool for machine learning education and research. It’s a Jupyter notebook environment that requires no setup to use as common packages have been  pre-installed. Google users can have access to a Tesla T4 GPU (approximately 15G memory). Note that when you connect to a GPU-based VM runtime, you are given a maximum of 12 hours at a time on the VM.

Colab is web-based, fast and convinient. You can simply upload this notebook and run it online. For the database needed in this task, you can download it and upload to colab OR you can save it in your google drive and link it with the colab.

It is convenient to upload local Jupyter Notebook files and data to Colab, please refer to the [tutorial](https://colab.research.google.com/notebooks/io.ipynb). 

In addition, Microsoft also provides the online platform [Azure Notebooks](https://notebooks.azure.com/help/introduction) for research of data science and machine learning, there are free trials for new users with credits.

# Introduction

Language models are very useful for a wide range of applications, e.g., speech recognition and machine translation. Consider a sentence consisting of words $x_1, x_2, …, x_m$, where $m$ is the length of the sentence, the goal of language modeling is to model the probability of the sentence, where $m \geq 1$, $x_i \in V $ and $V$ is the vocabulary of the corpus:
$$p(x_1, x_2, …, x_m)$$
In this project, we are going to explore both statistical language model and neural language model on the [Wikitext-2](https://blog.einstein.ai/the-wikitext-long-term-dependency-language-modeling-dataset/) datasets. Download wikitext-2 word-level data and put it under the ``data`` folder. There should be 3 data files in total: ``wiki.train.tokens``, ``wiki.valid.tokens`` and ``wiki.test.tokens``.

<font color=red> **(Please download the dataset before you proceed. Contact TA if you have trouble in this step.)** </font>

## Statistical  Language Model

A simple way is to view words as independent random variables (i.e., zero-th order Markovian assumption). The joint probability can be written as:
$$p(x_1, x_2, …, x_m)=\prod_{i=1}^m p(x_i)$$
However, this model ignores the word order information, to account for which, under the first-order Markovian assumption, the joint probability can be written as:
$$p(x_0, x_1, x_2, …, x_{m})= \prod_{i=1}^{m}p(x_i \mid x_{i-1})$$
Under the second-order Markovian assumption, the joint probability can be written as:
$$p(x_{-1}, x_0, x_1, x_2, …, x_{m})= \prod_{i=1}^{m}p(x_i \mid x_{i-2}, x_{i-1})$$
Similar to what we did in HMM, we will assume that $x_{-1}=START, x_0=START, x_{m} = STOP$ in this definition, where $START, STOP$ are special symbols referring to the start and the end of a sentence.







### Parameter estimation

Let's use $count(u)$ to denote the number of times the unigram $u$ appears in the corpus, use $count(v, u)$ to denote the number of times the bigram $v, u$ appears in the corpus, and $count(w, v, u)$ the times the trigram $w, v, u$ appears in the corpus, $u \in V \cup STOP$ and $w, v \in V \cup START$.

And the parameters of the unigram, bigram and trigram models can be obtained using maximum likelihood estimation (MLE).

- In the unigram model, the parameters can be estimated as: $$p(u) = \frac {count(u)}{c}$$, where $c$ is the total number of words in the corpus.
- In the bigram model, the parameters can be estimated as:
$$p(u \mid v) = \frac{count(v, u)}{count(v)}$$
- In the trigram model, the parameters can be estimated as:
$$p(u \mid w, v) = \frac{count(w, v, u)}{count(w, v)}$$




### Smoothing the parameters
#### Add-k Smoothing
Note, it is likely that many parameters of bigram and trigram models will be 0 because the relevant bigrams and trigrams involved do not appear in the corpus. If you don't have a way to handle these 0 probabilities, all the sentences that include such bigrams or trigrams will have probabilities of 0.

We'll use a Add-k Smoothing method to fix this problem, the smoothed parameters can be estimated as:

$$p_{add-k}(u)= \frac{count(u)+k}{c+k|V^*|}$$

$$p_{add-k}(u \mid v)= \frac{count(v, u)+k}{count(v)+k|V^*|}$$

$$p_{add-k}(u \mid w, v)= \frac{count(w, v, u)+k}{count(w, v)+k|V^*|}$$

where $k \in (0, 1)$ is the parameter of this approach, and $|V^*|$ is the size of the vocabulary $V^*$, here $V^*= V \cup STOP$. One way to choose the value of $k$ is by
optimizing the perplexity of the development set, namely to choose the value that minimizes the perplexity.
#### Interpolation
There is another way for smoothing which is named as **interpolation**. In interpolation, we always mix the probability estimates from
all the n-gram estimators, weighing and combining the trigram, bigram, and unigram counts. In simple linear interpolation, we combine different order n-grams by linearly interpolating all the models. Thus, we estimate the trigram probability $p(w_n|w_{n-2},w_{n-1})$ by mixing together the unigram, bigram, and trigram probabilities, each weighted by a $\lambda$:
$$\hat{p}(w_n|w_{n-2},w_{n-1}) = \lambda_1p(w_n|w_{n-2},w_{n-1})+\lambda_2p(w_n|w_{n-1})+\lambda_3p(w_n)$$
such that the $\lambda_s$ sum to 1:
$$\sum_i\lambda_i=1$$
In addition, $\lambda_1,\lambda_2,\lambda_3\geq 0$.

### Perplexity

Given a test set $D^{\prime}$ consisting of sentences $X^{(1)}, X^{(2)}, …, X^{(|D^{\prime}|)}$, each sentence $X^{(j)}$ consists of words $x_1^{(j)}, x_2^{(j)},…,x_{n_j}^{(j)}$, we can measure the probability of each sentence $X^{(j)}$, and the quality of the language model would be the probability it assigns to the entire set of test sentences, namely:
\begin{equation} 
\prod_{j=1}^{|D^{\prime}|}p(X^{(j)})
\end{equation}
Let's define average $log_2$ probability as:
\begin{equation} 
l=\frac{1}{c^{\prime}}\sum_{j=1}^{|D^{\prime}|}log_2p(X^{(j)})
\end{equation}
$c^{\prime}$ is the total number of words in the test set, $|D^{\prime}|$ is the number of sentences. And the perplexity is defined as:
\begin{equation} 
perplexity=2^{-l}
\end{equation}

The lower the perplexity, the better the language model.

# Questions

In [1]:
! pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio==0.8.1 -f https://download.pytorch.org/whl/torch_stable.html torchtext==0.9.1 nltk

from collections import Counter, namedtuple
import itertools
import numpy as np

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [2]:
with open('data/wikitext-2/wiki.train.tokens', 'r', encoding='utf8') as f:
    text = f.readlines()
    train_sents = [line.lower().strip('\n').split() for line in text]
    train_sents = [s for s in train_sents if len(s)>0 and s[0] != '=']

In [3]:
print(' '.join(train_sents[1][:10]))

the game began development in 2010 , carrying over a


In [4]:
### Test CUDA 11.1 working
import torch

print("Number of GPU: ", torch.cuda.device_count())
print("GPU Name: ", torch.cuda.get_device_name())


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Number of GPU:  1
GPU Name:  NVIDIA GeForce RTX 3060 Laptop GPU
Using device: cuda


c:\Users\atulp\anaconda3\envs\testenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Question 1.1 [code] **(4 points)** 
Implement the function **"compute_ngram"** that computes n-grams in the corpus.
 (Do not take the START and STOP symbols into consideration.) 

In [5]:
def compute_ngram(sents, n):
    '''
    Compute n-grams that appear in "sents".
    param:
        sents: list[list[str]] --- list of list of word strings
        n: int --- "n" gram
    return:
        ngram_set: set{str} --- a set of n-grams (no duplicate elements)
        ngram_dict: dict{ngram: counts} --- a dictionary that maps each ngram to its number occurence in "sents";
        This dict contains the parameters of our ngram model. E.g. if n=2, ngram_dict={('a','b'):10, ('b','c'):13}
        
        You may need to use "Counter", "tuple" function here.
    '''
    ngram_set = None
    ngram_dict = None
    ### YOUR CODE HERE
    ngram_dict = Counter()

    for sent in sents:
        ngram_dict.update([tuple(sent[i:i+n]) for i in range(len(sent)-n+1)])

    ngram_set = set(ngram_dict.keys())

    ### END OF YOUR CODE
    return ngram_set, ngram_dict

In [6]:
unigram_set, unigram_dict = compute_ngram(train_sents, 1)
print('unigram: %d' %(len(unigram_set)))
bigram_set, bigram_dict = compute_ngram(train_sents, 2)
print('bigram: %d' %(len(bigram_set)))
trigram_set, trigram_dict = compute_ngram(train_sents, 3)
print('trigram: %d' %(len(trigram_set)))

unigram: 28910
bigram: 577343
trigram: 1344047


### Question 1.2 [code] **(2 points)** 
List 5 most frequent unigrams, bigrams and trigrams as well as their counts.(Hint: use the built-in function .most_common in Counter class)  

In [7]:
# List 5 most frequent unigrams, bigrams and trigrams as well as their counts.
### YOUR CODE HERE
print('5 most frequent unigrams:')
print(unigram_dict.most_common(5))
print('5 most frequent bigrams:')
print(bigram_dict.most_common(5))
print('5 most frequent trigrams:')
print(trigram_dict.most_common(5))
### END OF YOUR CODE

5 most frequent unigrams:
[(('the',), 130519), ((',',), 99763), (('.',), 73388), (('of',), 56743), (('<unk>',), 53951)]
5 most frequent bigrams:
[(('of', 'the'), 17242), (('in', 'the'), 11778), ((',', 'and'), 11643), (('.', 'the'), 11274), ((',', 'the'), 8024)]
5 most frequent trigrams:
[((',', 'and', 'the'), 1393), ((',', '<unk>', ','), 950), (('<unk>', ',', '<unk>'), 901), (('one', 'of', 'the'), 866), (('<unk>', ',', 'and'), 819)]


### Question 2 [code] **(4 points)**
Now, we take the START and STOP symbols into consideration. So we need to pad the **train_sents** as described in "Statistical Language Model" before we apply "compute_ngram" function. For example, given a sentence "I like NLP", in a bigram model, we need to pad it as "START I like NLP STOP", in a trigram model, we need to pad it as "START START I like NLP STOP". For unigram model, it should be paded as "I like NLP STOP".

1. Implement the ``pad_sents`` function.
2. Pad ``train_sents``.
3. Apply ``compute_ngram`` function to these padded sents.
4. Implement ``ngram_prob`` function. Compute the probability for each n-gram in the variable **ngrams** according equations in **"Parameter estimation"**. List down the n-grams that have 0 probability. 



In [8]:
###############################################
ngrams = list()
with open('data/ngram.txt','r') as f:
    for line in f:
        ngrams.append(line.strip('\n').split())
print(ngrams)
###############################################

[['the'], ['the', 'computer'], ['go'], ['go', 'to'], ['have'], ['have', 'had'], ['and', 'the'], ['can'], ['can', 'sea'], ['a', 'number', 'of'], ['with', 'respect', 'to'], ['in', 'terms', 'of'], ['not', 'good', 'bad']]


In [9]:
START = '<START>'
STOP = '<STOP>'
###################################
def pad_sents(sents, n):
    '''
    Pad the sents according to n.
    params:
        sents: list[list[str]] --- list of sentences.
        n: int --- specify the padding type, 1-gram, 2-gram, or 3-gram.
    return:
        padded_sents: list[list[str]] --- list of padded sentences.
    '''
    #padded_sents = None
    padded_sents = []
    ### YOUR CODE HERE
    for sent in sents:
        padded_sent = [START] * (n - 1) + sent + [STOP]
        padded_sents.append(padded_sent)
    ### END OF YOUR CODE
    return padded_sents

In [10]:
uni_sents = pad_sents(train_sents, 1)
bi_sents = pad_sents(train_sents, 2)
tri_sents = pad_sents(train_sents, 3)

In [11]:
unigram_set, unigram_dict = compute_ngram(uni_sents, 1)
bigram_set, bigram_dict = compute_ngram(bi_sents, 2)
trigram_set, trigram_dict = compute_ngram(tri_sents, 3)

In [12]:
len(unigram_set),len(bigram_set),len(trigram_set)

(28911, 580825, 1363266)

In [13]:
num_words = sum([v for _,v in unigram_dict.items()])
print(num_words)

2024702


In [14]:
def ngram_prob(ngram, num_words, unigram_dic, bigram_dic, trigram_dic):
    '''
    params:
        ngram: list[str] --- a list that represents n-gram
        num_words: int --- total number of words
        unigram_dic: dict{ngram: counts} --- a dictionary that maps each 1-gram to its number of occurences in "sents";
        bigram_dic: dict{ngram: counts} --- a dictionary that maps each 2-gram to its number of occurence in "sents";
        trigram_dic: dict{ngram: counts} --- a dictionary that maps each 3-gram to its number occurence in "sents";
    return:
        prob: float --- probability of the "ngram"
    '''
    prob = None
    ### YOUR CODE HERE
    try:
        if len(ngram) == 1:
            prob = unigram_dic[tuple(ngram)] / num_words
        elif len(ngram) == 2:
            prob = bigram_dic[tuple(ngram)] / unigram_dic[tuple(ngram[:1])]
        elif len(ngram) == 3:
            prob = trigram_dic[tuple(ngram)] / bigram_dic[tuple(ngram[:2])]
    except ZeroDivisionError:
        prob = 0.0
    ### END OF YOUR CODE
    return prob

In [15]:
ngram_prob(ngrams[0], num_words,unigram_dict, bigram_dict, trigram_dict)

0.06446331361355893

In [16]:
### List down the n-grams that have 0 probability. 
### YOUR CODE HERE
zero_prob_ngrams = []
for ngram in ngrams:
    if ngram_prob(ngram, num_words, unigram_dict, bigram_dict, trigram_dict) == 0.0:
        zero_prob_ngrams.append(ngram)
print(zero_prob_ngrams)
### END OF YOUR CODE

[['can', 'sea'], ['not', 'good', 'bad']]


### Question 3 [code] **(4 points)**

1. Implement ``add_k_smoothing_ngram`` function to estimate ngram probability with ``add-k`` smoothing technique.
2. Implement ``interpolation_ngram`` function to estimate ngram probability with ``interpolation`` smoothing technique.
2. Implement ``perplexity`` function to compute the perplexity of the corpus "**valid_sents**" according to "**Perplexity**" section. The computation of $p(X^{(j)})$ depends on the n-gram model you choose.

In [17]:
with open('data/wikitext-2/wiki.valid.tokens', 'r', encoding='utf8') as f:
    text = f.readlines()
    valid_sents = [line.lower().strip('\n').split() for line in text]
    valid_sents = [s for s in valid_sents if len(s)>0 and s[0] != '=']

uni_valid_sents = pad_sents(valid_sents, 1)
bi_valid_sents = pad_sents(valid_sents, 2)
tri_valid_sents = pad_sents(valid_sents, 3)

In [18]:
def add_k_smoothing_ngram(ngram, k, num_words, unigram_dic, bigram_dic, trigram_dic):
    '''
    params:
        ngram: list[str] --- a list that represents n-gram
        k: float 
        num_words: int --- total number of words
        unigram_dic: dict{ngram: counts} --- a dictionary that maps each 1-gram to its number of occurences in "sents";
        bigram_dic: dict{ngram: counts} --- a dictionary that maps each 2-gram to its number of occurence in "sents";
        trigram_dic: dict{ngram: counts} --- a dictionary that maps each 3-gram to its number occurence in "sents";
    return:
        s_prob: float --- probability of the "ngram"
    '''
    s_prob = None
    V = len(unigram_dic)
    ### YOUR CODE HERE

    # Use the formula for add-k smoothing
    def add_k(n, d, k, V):
        return (n + k) / (d + k * V)
    
    try:
        if len(ngram) == 1:
            s_prob = add_k(unigram_dic[tuple(ngram)], num_words, k, V)
        elif len(ngram) == 2:
            s_prob = add_k(bigram_dic[tuple(ngram)], unigram_dic[tuple(ngram[:1])], k, V)
        elif len(ngram) == 3:
            s_prob = add_k(trigram_dic[tuple(ngram)], bigram_dic[tuple(ngram[:2])], k, V)        
    except:
        n = 0
        d = 0
    ### END OF YOUR CODE
    return s_prob

In [19]:
def interpolation_ngram(ngram, lam, num_words, unigram_dic, bigram_dic, trigram_dic):
    '''
    params:
        ngram: list[str] --- a list that represents n-gram
        lam: list[float] --- a list of length 3.lam[0], lam[1] and lam[2] are correspondence to trigram, bigram and unigram,repectively.
                             If len(ngram) == 1, lam[0]=lam[1]=0, lam[2]=1. If len(ngram) == 2, lam[0]=0. lam[0]+lam[1]+lam[2] = 1.
        num_words: int --- total number of words
        unigram_dic: dict{ngram: counts} --- a dictionary that maps each 1-gram to its number of occurences in "sents";
        bigram_dic: dict{ngram: counts} --- a dictionary that maps each 2-gram to its number of occurence in "sents";
        trigram_dic: dict{ngram: counts} --- a dictionary that maps each 3-gram to its number occurence in "sents";
    return:
        s_prob: float --- probability of the "ngram"
    '''
    s_prob = None
    ### YOUR CODE HERE
    if len(ngram) == 1:
        lam[0] = lam[1] = 0
        lam[2] = 1
        s_prob = lam[2] * ngram_prob(ngram, num_words, unigram_dic, bigram_dic, trigram_dic)
    elif len(ngram) == 2:
        lam[0] = 0
        s_prob = lam[1] * ngram_prob(ngram, num_words, unigram_dic, bigram_dic, trigram_dic) + lam[2] * ngram_prob(ngram[1:], num_words, unigram_dic, bigram_dic, trigram_dic)
    elif len(ngram) == 3:
        # Proportionately normalise the lambdas
        lam = [l / sum(lam) for l in lam]
        s_prob = lam[0] * ngram_prob(ngram, num_words, unigram_dic, bigram_dic, trigram_dic) + lam[1] * ngram_prob(ngram[1:], num_words, unigram_dic, bigram_dic, trigram_dic) + lam[2] * ngram_prob(ngram[2:], num_words, unigram_dic, bigram_dic, trigram_dic)
    ### END OF YOUR CODE
    return s_prob

In [20]:
add_k_prob = add_k_smoothing_ngram(ngrams[5], 0.01, num_words, unigram_dict, bigram_dict, trigram_dict)
interpolation_prob = interpolation_ngram(ngrams[5], [0.6,0.3,0.1], num_words, unigram_dict, bigram_dict, trigram_dict)
print(ngrams[5])
print(add_k_prob, interpolation_prob)

['have', 'had']
0.011710673716309527 0.004086999802140221


In [21]:
def perplexity(n, method, num_words, valid_sents, unigram_dic, bigram_dic, trigram_dic, k=0, lam=[0,0,1]):
#     params:
#         n: int --- n-gram model you choose 
#         method: int ---- method == 0, use add_k_smoothing; method != 0, use interpolation method.
#         num_words: int --- total number of words
#         valid_sents: list[list[str]] --- list of sentences
#         unigram_dic: dict{ngram: counts} --- a dictionary that maps each 1-gram to its number of occurences in "sents";
#         bigram_dic: dict{ngram: counts} --- a dictionary that maps each 2-gram to its number of occurence in "sents";
#         trigram_dic: dict{ngram: counts} --- a dictionary that maps each 3-gram to its number occurence in "sents";
#         k: float --- The parameter of add_k_smoothing
#         lam: list[float] --- a list of length 3. The parameter of interpolation. 
#    return:
#         ppl: float --- perplexity of valid_sents
#     '''
#     '''
    ppl = None
    ### YOUR CODE HERE
    prob = 0
    if method == 0:
      for i in valid_sents:
        start=0
        for j in range(n,len(i)):
            prob+=np.log2(add_k_smoothing_ngram(i[start:j], k, num_words, unigram_dict, bigram_dict, trigram_dict))
            start+=1
            # Handle div by 0
            if prob == float('-inf'):
                prob = 0
    else:
      for i in valid_sents:
        start = 0
        for j in range(n,len(i)):
            prob += np.log2(interpolation_ngram(i[start:j], lam, num_words, unigram_dict, bigram_dict, trigram_dict))
            start+=1
            # Handle div by 0
            if prob == float('-inf'):
                prob = 0
    ppl=2**(-1*prob/np.sum([len(i) for i in valid_sents]))
    ### END OF YOUR CODE
    return ppl

In [22]:
perplexity(1, 0, num_words, uni_valid_sents, unigram_dict, bigram_dict, trigram_dict, k=0.1, lam=[0,0,1])

799.6619443069386

In [23]:
perplexity(1, 0, num_words, uni_valid_sents, unigram_dict, bigram_dict, trigram_dict, k=0.1, lam=[0,0,1])

799.6619443069386

### Question 4 [code][written] **(4 points)**
1. Based on add-k smoothing method, try out different $k\in [ 0.0001, 0.001, 0.01, 0.1, 0.5]$ and different n-gram model (unigram, bigram and trigram). Find the model and $k$ that gives the best perplexity on "**valid_sents**" (smaller is better).
2. Based on interpolation method, try out different $\lambda$ where $\lambda_1 = \lambda_2$ and $\lambda_3\in [0.1, 0.2, 0.4, 0.6, 0.8]$. Find the $\lambda$ that gives the best perplexity on "**valid_sents**" (smaller is better).
3. Based on the methods and parameters we provide, choose the method that peforms best on the validation data.

In [24]:
n = [1,2,3]
k = [0.0001, 0.001, 0.01, 0.1, 0.5]

best_ppl = float('inf')
best_nk = {'n':0,'k':0}
### YOUR CODE HERE (add-k smoothing method)

for n_val in n:
    for k_val in k:
        ppl = perplexity(n_val, 0, num_words, uni_valid_sents, unigram_dict, bigram_dict, trigram_dict, k_val, lam=[0,0,1])
        if ppl < best_ppl:
            best_ppl = ppl
            best_nk = {'n': n_val, 'k': k_val}
            print("Best values so far: ", best_ppl, best_nk)

### END OF YOUR CODE
print(best_ppl, best_nk)


Best values so far:  799.4969893037119 {'n': 1, 'k': 0.0001}
Best values so far:  660.191107247087 {'n': 2, 'k': 0.0001}
Best values so far:  481.1084956934896 {'n': 2, 'k': 0.001}
Best values so far:  478.36441587832854 {'n': 2, 'k': 0.01}
478.36441587832854 {'n': 2, 'k': 0.01}


In [25]:
lambda_3 = [0.1, 0.2, 0.4, 0.6, 0.8]

best_ppl = float('inf')
best_nk = {'lambda':0}
### YOUR CODE HERE (interpolation method)

for lam_val in lambda_3:
    ppl = perplexity(3, 1, num_words, uni_valid_sents, unigram_dict, bigram_dict, trigram_dict, k=0.1, lam=[(1 - lam_val) / 2,(1 - lam_val) / 2, lam_val])
    if ppl <= best_ppl:
        best_ppl = ppl
        best_nk = {'lambda': lam_val}
        print("Best values so far: ", best_ppl, best_nk)

### END OF YOUR CODE
print(best_ppl, best_nk)

Best values so far:  312.40888050539996 {'lambda': 0.1}
Best values so far:  282.2676598367113 {'lambda': 0.2}
Best values so far:  271.68061888232074 {'lambda': 0.4}
271.68061888232074 {'lambda': 0.4}


Based on the methods and parameters we provide, choose the method that peforms best on the validation data (**write your answer**): 

The best performing model is the one with the lowest perplexity on the validation data. In this case, it is the trigram model using interpolation smoothing, with lambda being 0.4. I suspect this is because it's the closest to having equal weightages for the unigram, bigram and trigram models when the third lambda value equals 0.4.

### Question 5 [code] **(4 points)**

Evaluate the perplexity of the test data **test_sents** based on the best model you choose in **Question 4**.

In [26]:
with open('data/wikitext-2/wiki.test.tokens', 'r', encoding='utf8') as f:
    text = f.readlines()
    test_sents = [line.lower().strip('\n').split() for line in text]
    test_sents = [s for s in test_sents if len(s)>0 and s[0] != '=']

uni_test_sents = pad_sents(test_sents, 1)
bi_test_sents = pad_sents(test_sents, 2)
tri_test_sents = pad_sents(test_sents, 3)

In [27]:
### YOUR CODE HERE
# Ideal lambda value is 0.4
lam_val = 0.4

ppl = perplexity(3, 1, num_words, uni_test_sents, unigram_dict, bigram_dict, trigram_dict, k=0.1, lam=[(1 - lam_val) / 2,(1 - lam_val) / 2, lam_val])
print("Perplexity on test set with unigram: ", ppl)
ppl = perplexity(3, 1, num_words, bi_test_sents, unigram_dict, bigram_dict, trigram_dict, k=0.1, lam=[(1 - lam_val) / 2,(1 - lam_val) / 2, lam_val])
print("Perplexity on test set with bigram: ", ppl)
ppl = perplexity(3, 1, num_words, tri_test_sents, unigram_dict, bigram_dict, trigram_dict, k=0.1, lam=[(1 - lam_val) / 2,(1 - lam_val) / 2, lam_val])
print("Perplexity on test set with trigram: ", ppl)
### END OF YOUR CODE

Perplexity on test set with unigram:  255.223922904202
Perplexity on test set with bigram:  256.294503525543
Perplexity on test set with trigram:  259.35022996725246


## Neural Language Model


<img src="bilstm.png" alt="drawing" style="width:500px;"/>

We will create a LSTM language model as shown in figure and train it on the Wikitext-2 dataset. 
The data generators (train\_iter, valid\_iter, test\_iter) have been provided. 
The word embeddings together with the parameters in the LSTM model will be learned from scratch.

[Pytorch](https://pytorch.org/tutorials/) and [torchtext](https://torchtext.readthedocs.io/en/latest/index.html#) are required in this part. Do not make any changes to the provided code unless you are requested to do so. 

### Question 6 [code] **(10 points)**
- Implement the ``__init__`` function in ``LangModel`` class.
- Implement the ``forward`` function in ``LangModel`` class.
- Complete the training code in ``train`` function.
    Then complete the testing code in  ``test`` function and 
    compute the perplexity of the test data ``test_iter``. The test perplexity should be below 150.

In [28]:
import torchtext
import torch
import torch.nn.functional as F
from torchtext.legacy.datasets import WikiText2
from torch import nn, optim
from torchtext.legacy import data
from nltk import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
torch.manual_seed(222)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\atulp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\atulp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [29]:
def tokenizer(text):
    '''Tokenize a string to words'''
    return word_tokenize(text)

START = '<START>'
STOP = '<STOP>'
#Load and split data into three parts
TEXT = data.Field(lower=True, tokenize=tokenizer, init_token=START, eos_token=STOP)
train, valid, test = WikiText2.splits(TEXT) 

In [30]:
#Build a vocabulary from the train dataset
TEXT.build_vocab(train)
print('Vocabulary size:', len(TEXT.vocab))

Vocabulary size: 28907


In [31]:
BATCH_SIZE = 64
# the length of a text feeding to the RNN layer
BPTT_LEN = 32           
# train, validation, test data
train_iter, valid_iter, test_iter = data.BPTTIterator.splits((train, valid, test),
                                                                batch_size=BATCH_SIZE,
                                                                bptt_len=BPTT_LEN,
                                                                repeat=False)

In [32]:
#Generate a batch of train data
batch = next(iter(train_iter))
text, target = batch.text, batch.target
print('Size of text tensor',text.size())
print('Size of target tensor',target.size())

Size of text tensor torch.Size([32, 64])
Size of target tensor torch.Size([32, 64])


In [33]:
class LangModel(nn.Module):
    def __init__(self, lang_config):
        super(LangModel, self).__init__()
        self.vocab_size = lang_config['vocab_size']
        self.emb_size = lang_config['emb_size']
        self.hidden_size = lang_config['hidden_size']
        self.num_layer = lang_config['num_layer']
        self.bidirectional = lang_config['bidirectional']
        
        self.embedding = None
        self.lstm = None
        self.linear = None
        
        ### TODO: 
        ###    1. Initialize 'self.embedding' with nn.Embedding function and 2 variables we have initialized for you
        ###    2. Initialize 'self.lstm' with nn.LSTM function and 3 variables we have initialized for you
        ###    3. Initialize 'self.linear' with nn.Linear function and 2 variables we have initialized for you
        ### Reference:
        ###        https://pytorch.org/docs/stable/nn.html
        
        ### YOUR CODE HERE (3 lines)
        self.embedding = nn.Embedding(self.vocab_size, self.emb_size)
        self.lstm = nn.LSTM(self.emb_size, self.hidden_size, self.num_layer, bidirectional=self.bidirectional)
        self.linear = nn.Linear(self.hidden_size * (2 if self.bidirectional else 1), self.vocab_size)

        ### END OF YOUR CODE
        
    def forward(self, batch_sents, hidden=None):
        '''
        params:
            batch_sents: torch.LongTensor of shape (sequence_len, batch_size)
        return:
            normalized_score: torch.FloatTensor of shape (sequence_len, batch_size, vocab_size)
        '''
        normalized_score = None
        hidden = hidden
        ### TODO:
        ###      1. Feed the batch_sents to self.embedding  
        ###      2. Feed the embeddings to self.lstm. Remember to pass "hidden" into self.lstm, even if it is None. But we will 
        ###         use "hidden" when implementing greedy search.
        ###      3. Apply linear transformation to the output of self.lstm
        ###      4. Apply 'F.log_softmax' to the output of linear transformation
        ###
        ### YOUR CODE HERE (4 lines)
        emb = self.embedding(batch_sents)
        lstm_out, hidden = self.lstm(emb, hidden)
        linear_out = self.linear(lstm_out)
        normalized_score = F.log_softmax(linear_out, dim=2)
        
        ### END OF YOUR CODE
        return normalized_score, hidden

In [34]:
def train(model, train_iter, valid_iter, vocab_size, criterion, optimizer, num_epochs):
    for n in range(num_epochs):
        train_loss = 0
        target_num = 0
        model.train()
        for batch in train_iter:
            
            text, targets = batch.text.to(device), batch.target.to(device)
            loss = None
            
            ### we don't consider "hidden" here. So according to the default setting, "hidden" will be None
            ### YOU CODE HERE (~5 lines)
            prediction,_ = model(text)
            loss = criterion(prediction.view(-1, vocab_size), targets.view(-1))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            ### END OF YOUR CODE
            ##########################################
            train_loss += loss.item() * targets.size(0) * targets.size(1)
            target_num += targets.size(0) * targets.size(1)

        train_loss /= target_num

        # monitor the loss of all the predictions
        val_loss = 0
        target_num = 0
        model.eval()
        for batch in valid_iter:
            text, targets = batch.text.to(device), batch.target.to(device)
            
            prediction,_ = model(text)
            loss = criterion(prediction.view(-1, vocab_size), targets.view(-1))
            
            val_loss += loss.item() * targets.size(0) * targets.size(1)
            target_num += targets.size(0) * targets.size(1)
        val_loss /= target_num

        print('Epoch: {}, Training Loss: {:.4f}, Validation Loss: {:.4f}'.format(n+1, train_loss, val_loss))   

In [35]:
def test(model, vocab_size, criterion, test_iter):
    '''
    params: 
        model: LSTM model
        test_iter: test data
    return:
        ppl: perplexity 
    '''
    ppl = None
    test_loss = 0
    target_num = 0
    with torch.no_grad():
        for batch in test_iter:
            text, targets = batch.text.to(device), batch.target.to(device)

            prediction,_ = model(text)
            loss = criterion(prediction.view(-1, vocab_size), targets.view(-1))

            test_loss += loss.item() * targets.size(0) * targets.size(1)
            target_num += targets.size(0) * targets.size(1)

        test_loss /= target_num
        
        ### Compute perplexity according to "test_loss"
        ### Hint: Consider how the loss is computed.
        ### YOUR CODE HERE(1 line)
        ppl = np.exp(test_loss)
        
        ### END OF YOUR CODE
        return ppl

In [36]:
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vocab_size = len(TEXT.vocab)

In [37]:
config = {
    'vocab_size':vocab_size,
    'emb_size':128,
    'hidden_size':128,
    'num_layer':1,
    'bidirectional': False
}

LM = LangModel(config)
LM = LM.to(device)

criterion = nn.NLLLoss(reduction='mean')
optimizer = optim.Adam(LM.parameters(), lr=1e-3, betas=(0.7, 0.99))

In [38]:
train(LM, train_iter, valid_iter, vocab_size, criterion, optimizer, num_epochs)

Epoch: 1, Training Loss: 6.0576, Validation Loss: 5.1698
Epoch: 2, Training Loss: 5.3879, Validation Loss: 4.9413
Epoch: 3, Training Loss: 5.1198, Validation Loss: 4.8540
Epoch: 4, Training Loss: 4.9521, Validation Loss: 4.8109
Epoch: 5, Training Loss: 4.8314, Validation Loss: 4.7835
Epoch: 6, Training Loss: 4.7346, Validation Loss: 4.7642
Epoch: 7, Training Loss: 4.6527, Validation Loss: 4.7528
Epoch: 8, Training Loss: 4.5825, Validation Loss: 4.7459
Epoch: 9, Training Loss: 4.5212, Validation Loss: 4.7448
Epoch: 10, Training Loss: 4.4667, Validation Loss: 4.7471


In [39]:
test(LM, vocab_size, criterion, test_iter)

99.41832661548061

In [40]:
config = {
    'vocab_size':vocab_size,
    'emb_size':128,
    'hidden_size':128,
    'num_layer':1,
    'bidirectional': True
}

biLSTM = LangModel(config)
biLSTM = biLSTM.to(device)

criterion = nn.NLLLoss(reduction='mean')
optimizer = optim.Adam(biLSTM.parameters(), lr=1e-3, betas=(0.7, 0.99))

In [41]:
train(biLSTM, train_iter, valid_iter, vocab_size, criterion, optimizer, num_epochs)

Epoch: 1, Training Loss: 3.1737, Validation Loss: 1.3003
Epoch: 2, Training Loss: 0.9979, Validation Loss: 0.6369
Epoch: 3, Training Loss: 0.5144, Validation Loss: 0.4420
Epoch: 4, Training Loss: 0.3227, Validation Loss: 0.3510
Epoch: 5, Training Loss: 0.2297, Validation Loss: 0.3055
Epoch: 6, Training Loss: 0.1834, Validation Loss: 0.2808
Epoch: 7, Training Loss: 0.1582, Validation Loss: 0.2677
Epoch: 8, Training Loss: 0.1423, Validation Loss: 0.2609
Epoch: 9, Training Loss: 0.1309, Validation Loss: 0.2564
Epoch: 10, Training Loss: 0.1214, Validation Loss: 0.2537


In [42]:
test(biLSTM, vocab_size, criterion, test_iter)

1.2749883700175868

### Question 7 [code][written] **(8 points)**
- We will use the hidden vectors (the working memory) of LSTM as the contextual embeddings. Implement ``contextual_embedding`` function.
- Use the ``contextual_embedding`` function to get the contextual embeddings of the word "play" in three sequences "to play", "dance play" and "sing play". Then calculate the cosine similarity of "play" from each pair of sequences "to play", "dance play" and "sing play". Assume that $\boldsymbol{w}_1$ and $\boldsymbol{w}_2$ are embeddings of "play" in sequences "to play" and "dance play" respectively. The cosine similarity can be calculated as 
\begin{align}
similarity = cos(\theta) = \frac{\boldsymbol{w}^{\rm T}_1\boldsymbol{w}_2}{||\boldsymbol{w}_1||_2||\boldsymbol{w}_2||_2}
\end{align}
Give the explanation of the results. 

In [43]:
def contextual_embedding(model, sentence):
    '''
    params: 
        model: LSTM model
        sentence -- list[str]: list of tokens, e.g., ['I', 'am',...]
    return:
        embeddings -- numpy array of shape (length of sentence, word embedding size)
    '''
    model.eval()
    hidden = None
    
    ### YOUR CODE HERE

    ### END OF YOUR CODE
    embeddings = np.zeros((0,128))
    for idx, i in enumerate(sentence):
        ID = TEXT.vocab.stoi[i]
        out, hidden = model(torch.LongTensor([[ID]]).to(device), hidden)
        embeddings = np.append(embeddings, hidden[0].cpu().detach().numpy()[0],axis=0)  
    return embeddings

In [44]:
sink_seq1 = "wood does not sink in water" # sink is in index 3
sink_seq2 = "a small water leak will sink the ship" # sink is in index 5
sink_seq3 = "there are plates in the kitchen sink" # sink is in the last index
sink_seq4 = "the kitchen sink was full of dirty dishes" # sink is in index 2

### YOUR CODE HERE
def cosine(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

# Each sentence's embeddings for sink
# standard LSTM
stan_seq1 = contextual_embedding(LM, sink_seq1.split())[3]
stan_seq2 = contextual_embedding(LM, sink_seq2.split())[5]
stan_seq3 = contextual_embedding(LM, sink_seq3.split())[-1]
stan_seq4 = contextual_embedding(LM, sink_seq4.split())[2]

# bidirectional LSTM
bidi_seq1 = contextual_embedding(biLSTM, sink_seq1.split())[3]
bidi_seq2 = contextual_embedding(biLSTM, sink_seq2.split())[5]
bidi_seq3 = contextual_embedding(biLSTM, sink_seq3.split())[-1]
bidi_seq4 = contextual_embedding(biLSTM, sink_seq4.split())[2]

# Calculate cosine similarities for each sink embedding, restrict to standard and bidirectional LSTM
# Basically all permutations of the sink embeddings but don't mix between standard and bidirectional
results = {}
for i in range(1, 5):
    for j in range(1, 5):
        if i == j:
            continue
        results[(i, j)] = cosine(locals()['stan_seq' + str(i)], locals()['stan_seq' + str(j)]), cosine(locals()['bidi_seq' + str(i)], locals()['bidi_seq' + str(j)])

for k, v in results.items():
    print(f"Standard LSTM: {k} - {v[0]}, Bidirectional LSTM: {k} - {v[1]}")
### END OF YOUR CODE


Standard LSTM: (1, 2) - 0.7883657376138035, Bidirectional LSTM: (1, 2) - 0.5561252451627524
Standard LSTM: (1, 3) - 0.5877902958178445, Bidirectional LSTM: (1, 3) - 0.4479758609576928
Standard LSTM: (1, 4) - 0.5768970044671943, Bidirectional LSTM: (1, 4) - 0.35463701236150924
Standard LSTM: (2, 1) - 0.7883657376138035, Bidirectional LSTM: (2, 1) - 0.5561252451627524
Standard LSTM: (2, 3) - 0.6284939673226192, Bidirectional LSTM: (2, 3) - 0.6285522043522962
Standard LSTM: (2, 4) - 0.5742108603788779, Bidirectional LSTM: (2, 4) - 0.5485594132365814
Standard LSTM: (3, 1) - 0.5877902958178445, Bidirectional LSTM: (3, 1) - 0.4479758609576928
Standard LSTM: (3, 2) - 0.6284939673226192, Bidirectional LSTM: (3, 2) - 0.6285522043522962
Standard LSTM: (3, 4) - 0.8601641905526018, Bidirectional LSTM: (3, 4) - 0.7417703338013901
Standard LSTM: (4, 1) - 0.5768970044671943, Bidirectional LSTM: (4, 1) - 0.35463701236150924
Standard LSTM: (4, 2) - 0.5742108603788779, Bidirectional LSTM: (4, 2) - 0.548

***write your explanation:***



As bidirectional LSTMs account for both forward and backward directions when calculating word embeddings in comparison to standard LSTMs, they have a better understanding of the context in which a word appears. As a result of this, the similarities between the cosines are smaller than those of the uni-directional LSTM. The contexts in sentences 1-2 and sentences 3-4 are completely different for the word "sink" (initial case being the verb and second case being the noun), which is why in the bi-directional LSTM the cosine similarity is lower. In the uni-directional LSTM, it merely accounts for the forward direction, not providing as much context for the word "sink" as the bi-directional LSTM does, hence having a higher cosine similarity across all pairs of sentences.

### Requirements:
- This is an individual project. 
- Write down names and IDs of students with whom you have discussed (if any). 
- You should **NOT** copy other's answer, once discovered, the person will get **0** in this mini project.
- Complete answers and Python code in the ``mini_project.ipynb`` file. 
- Follow the honor code strictly.
- Submit the file before the due on eDimension system.